In [1]:
import os

# os.environ['GITHUB_ACCESS_TOKEN'] = ''

# print(os.getenv("GITHUB_ACCESS_TOKEN", None))

from lean_dojo import *

/home/mcwave/anaconda3/envs/lean3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-07 19:59:27,757	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
repo = LeanGitRepo(
    "https://github.com/leanprover-community/mathlib3",
    "65a1391a0106c9204fe45bc73a039f056558cb83",
)

repo

LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83')

In [3]:
from utils.lean_math_utils import *

theorem = Theorem(repo, 'src/algebra/algebra/basic.lean', "algebra.left_comm")

print("Entering")
dojo, state_0 = Dojo(theorem).__enter__()
print("Done")

state_1 = dojo.run_tac(state_0, "rw [← mul_assoc]")

print("STATE 1:\n", state_1.pp)
classify_lean_elements(state_1.pp)

state_2 = dojo.run_tac(state_1, "rw [← commutes]") 
print("STATE 2:\n", state_2.pp)

state_3 = dojo.run_tac(state_2, "rw [mul_assoc]") 
print("STATE 3:\n", state_3)

dojo.__exit__(None, None, None)

Entering


2024-09-07 19:59:38.478 | WARNING  | lean_dojo.interaction.dojo:__enter__:227 - Docker is strongly recommended when using LeanDojo with Lean 3. See https://leandojo.readthedocs.io/en/latest/user-guide.html#advanced-running-within-docker.


Done
STATE 1:
 R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ x * ⇑(algebra_map R A) r * y = ⇑(algebra_map R A) r * (x * y)
STATE 2:
 R : Type u,
A : Type w,
_inst_1 : comm_semiring R,
_inst_3 : semiring A,
_inst_4 : algebra R A,
x : A,
r : R,
y : A
⊢ ⇑(algebra_map R A) r * x * y = ⇑(algebra_map R A) r * (x * y)
STATE 3:
 ProofFinished(tactic_state_id=3, message='')


In [5]:
import pickle

results = []
fin = open('/home/mcwave/code/axiomatization/lean3/datasets/provability/rag/src__ring_theory__ring_invo.lean.pkl', 'rb')

while True:
    try:
        result = pickle.load(fin)
    except Exception as e:
        print(e)
        break
    file_path, full_name, theorem, state_pair = result
    results.append(result)

fin.close()

print(len(results), "states loaded")

theorems = set()
for file_path, full_name, theorem, state_pair in results:
    if state_pair is not None:
        theorems.add(full_name)
print("Theorems:")
for full_name in theorems:
    print(full_name)

Ran out of input
9607 states loaded
Theorems:
ring_invo.map_eq_zero_iff
ring_invo.involution
ring_invo.to_fun_eq_coe
ring_invo.coe_ring_equiv


In [6]:
SELECTED_THEOREM = 'ring_invo.map_eq_zero_iff'

def is_pp_useful(pp):
    lines = pp.split('\n')
    targets = []
    conditions = []
    min_freq = 1000000
    for line in lines:
        if line.startswith("⊢"):
            target = line.strip()
            targets.append(target)
        else:
            conditions.append(line.strip())
    if '⊢ False' in targets:
        return False
    if '⊢ R' in targets:
        return False
    if '?m.' in pp:
        return False
    return True

num_output = 0
for i in range(len(results)):
    file_path, full_name, theorem, state_pair = results[i]
    if full_name != SELECTED_THEOREM: #'IsCoprime.of_mul_right_left': #'IsCoprime.mul_right':
        continue
    if not is_pp_useful(results[i][3][0].pp):
        continue
    print("CASE", i)
    print(results[i][3][0].pp)
    print(theorem)
    print()
    num_output += 1
    if num_output > 400:
        break

CASE 5235
R : Type u_1,
_inst_1 : semiring R,
x : R,
nvar0 : ring_invo R
⊢ ⇑nvar0 x = 0 ↔ x = 0
Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/ring_theory/ring_invo.lean'), full_name='ring_invo.map_eq_zero_iff')

CASE 5236
R : Type u_1,
_inst_1 : semiring R,
x : R
⊢ ∀ (f : ring_invo R), ⇑f x = 0 ↔ x = 0
Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/ring_theory/ring_invo.lean'), full_name='ring_invo.map_eq_zero_iff')

CASE 5237
R : Type u_1,
_inst_1 : semiring R,
f : ring_invo R,
x : R
⊢ ⇑f x = 0 ↔ x = 0
Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/ring_theory/ring_invo.lean'), full_name='ring_invo.map_eq_zero_iff')

CASE 5238
R : Type u_1,
_inst_1 : semiri

In [7]:
result_idx = 5236

print(results[result_idx])

print("\nTactics:")
for tactic in results[result_idx][-1][-1]:
    print(tactic)

('src/ring_theory/ring_invo.lean', 'ring_invo.map_eq_zero_iff', Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib3', commit='65a1391a0106c9204fe45bc73a039f056558cb83'), file_path=PosixPath('src/ring_theory/ring_invo.lean'), full_name='ring_invo.map_eq_zero_iff'), (TacticState(pp='R : Type u_1,\n_inst_1 : semiring R,\nx : R\n⊢ ∀ (f : ring_invo R), ⇑f x = 0 ↔ x = 0', id=7, message=''), ProofFinished(tactic_state_id=1384, message=''), 5, ['rw [← mul_zero]', 'contradiction', 'exact f', 'exact λ a b, ne.symm', 'subst nvar0']))

Tactics:
rw [← mul_zero]
contradiction
exact f
exact λ a b, ne.symm
subst nvar0


In [8]:
from utils.lean_math_utils import *

theorem = Theorem(repo, 'src/ring_theory/ring_invo.lean', "ring_invo.map_eq_zero_iff")

print("Entering")
dojo, state_0 = Dojo(theorem).__enter__()
print("Done")

print(state_0.pp)

Entering


2024-09-07 20:06:34.166 | WARNING  | lean_dojo.interaction.dojo:__enter__:227 - Docker is strongly recommended when using LeanDojo with Lean 3. See https://leandojo.readthedocs.io/en/latest/user-guide.html#advanced-running-within-docker.


Done
R : Type u_1,
_inst_1 : semiring R,
f : ring_invo R,
x : R
⊢ ⇑f x = 0 ↔ x = 0


In [9]:
state_1 = dojo.run_tac(state_0, "rw [← mul_zero]")

print("STATE 1:\n", state_1.pp)

STATE 1:
 R : Type u_1,
_inst_1 : semiring R,
f : ring_invo R,
x : R
⊢ ⇑f x = ?m_1 * 0 ↔ x = 0

R : Type u_1,
_inst_1 : semiring R,
f : ring_invo R,
x : R
⊢ Rᵐᵒᵖ


In [11]:
state_2 = dojo.run_tac(state_1, "exact f")

print("STATE 2:\n", state_2.pp)

AttributeError: 'LeanError' object has no attribute 'pp'

In [12]:
dojo.__exit__(None, None, None)